In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import networkx as nx
import osmnx as ox
import os
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import Polygon
import shapely.ops as so
import pandas as pd
import numpy as np

import importlib

In [2]:
place = {"city": "Singapore", "country": "Singapore"}
tags = {"amenity": True}
gdf = ox.features_from_place(place, tags)

In [4]:
gdf['amenity'].unique()

array(['parking_entrance', 'parking', 'toilets', 'fast_food', 'cafe',
       'food_court', 'bank', 'post_office', 'place_of_worship',
       'restaurant', 'shelter', 'bench', 'atm', 'bus_station', 'college',
       'telephone', 'fuel', 'bar', 'pub', 'pharmacy', 'taxi', 'theatre',
       'school', 'bicycle_parking', 'cinema', 'clock', 'fire_station',
       'vending_machine', 'dentist', 'post_box', 'police', 'recycling',
       'ferry_terminal', 'library', 'fountain', 'brothel', 'kindergarten',
       'bbq', 'waste_basket', 'community_centre', 'drinking_water',
       'clinic', 'marketplace', 'waste_disposal', 'bar;restaurant',
       'shower', 'grave_yard', 'townhall', 'doctors', 'bicycle_rental',
       'ice_cream', 'nightclub', 'yes', 'university', 'arts_centre',
       'prep_school', 'social_facility', 'events_venue', 'veterinary',
       'casino', 'country_club', 'photo_booth', 'reception_desk',
       'conference_centre', 'admin', 'public_building', 'car_rental',
       'reception

In [19]:
amenity_df = gdf[gdf['amenity'].isin(['school', 'university', 'college','polytechnic','institute'])]
amenity_df

geometry  \
element id                                                              
node    410477196                           POINT (103.95753 1.35645)   
        521081225                            POINT (103.84038 1.3028)   
        1703872641                          POINT (103.86969 1.38727)   
        1703872645                          POINT (103.86962 1.38727)   
        1800164471                          POINT (103.83793 1.33863)   
...                                                               ...   
way     1255355512  POLYGON ((103.75518 1.38197, 103.75625 1.38158...   
        1266704541  POLYGON ((103.79014 1.30422, 103.79023 1.30405...   
        1301675688  POLYGON ((103.87736 1.36354, 103.87827 1.36349...   
        1316079408  POLYGON ((103.85796 1.37323, 103.85825 1.37259...   
        1329131232  POLYGON ((103.81757 1.4576, 103.81721 1.45711,...   

                       amenity parking source created_by  \
element id                                                 
node    410477196      college     NaN    NaN        NaN   
        521081225       school     NaN    NaN        NaN   
        1703872641      school     NaN    NaN        NaN   
        1703872645      school     NaN    NaN        NaN   
        1800164471      school     NaN    NaN        NaN   
...                        ...     ...    ...        ...   
way     1255355512      school     NaN    NaN        NaN   
        1266704541  university     NaN    NaN        NaN   
        1301675688      school     NaN    NaN        NaN   
        1316079408      school     NaN    NaN        NaN   
        1329131232      school     NaN    NaN        NaN   

                                                        name name:it religion  \
element id                                                                      
node    410477196                                    Blk 390     NaN      NaN   
        521081225                                 ACC EduHub     NaN      NaN   
        1703872641                       Hua Language Centre     NaN      NaN   
        1703872645        Jan & Elly English Language School     NaN      NaN   
        1800164471                                       NaN     NaN      NaN   
...                                                      ...     ...      ...   
way     1255355512    Former Teck Whye Primary School Campus     NaN      NaN   
        1266704541                             NTU@one-north     NaN      NaN   
        1301675688  Idea Minds Employment Development Centre     NaN      NaN   
        1316079408                      APSN Chaoyang School     NaN      NaN   
        1329131232        Rainbow Centre Admiral Hill School     NaN      NaN   

                   operator  bus  ... short_name:en short_name:es  \
element id                        ...                               
node    410477196       NaN  yes  ...           NaN           NaN   
        521081225       NaN  NaN  ...           NaN           NaN   
        1703872641      NaN  NaN  ...           NaN           NaN   
        1703872645      NaN  NaN  ...           NaN           NaN   
        1800164471      NaN  NaN  ...           NaN           NaN   
...                     ...  ...  ...           ...           ...   
way     1255355512      NaN  NaN  ...           NaN           NaN   
        1266704541      NaN  NaN  ...           NaN           NaN   
        1301675688      NaN  NaN  ...           NaN           NaN   
        1316079408      NaN  NaN  ...           NaN           NaN   
        1329131232      NaN  NaN  ...           NaN           NaN   

                   short_name:krj short_name:la short_name:pag short_name:pam  \
element id                                                                      
node    410477196             NaN           NaN            NaN            NaN   
        521081225             NaN           NaN            NaN            NaN   
        1703872641            NaN           NaN       

In [34]:
# check if a character is contained in the column
schools_df = amenity_df[amenity_df['name'].str.contains('International|University|Polytechnic|College|ITE|junior college|primary|secondary', na=False, case=False, regex=True)]
print(len(schools_df.index))
# get the centroid coordinates from the geometry column
schools_centroid = schools_df['geometry'].centroid
# create columns latitude and longitude from centroid geometry
schools_df['latitude'] = schools_centroid.y
schools_df['longitude'] = schools_centroid.x
schools_df = schools_df[['name', 'amenity','latitude', 'longitude']]
save_dir = r"C:\Users\hypak\OneDrive - Singapore Management University\Documents - Heat Risk Index Development\Data\Schools"
schools_df.to_csv(os.path.join(save_dir, 'OSM_schools.csv'), index=False)

378


C:\Users\hypak\AppData\Local\Temp\ipykernel_2372\1785045440.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  schools_centroid = schools_df['geometry'].centroid
c:\Users\hypak\AppData\Local\miniforge3\envs\ox\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\hypak\AppData\Local\miniforge3\envs\ox\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume